In [176]:
from parse_rules import *
import numpy as np
import pandas as pd
from py2neo import Node,Relationship,Graph,Path,Subgraph
from py2neo import NodeMatcher,RelationshipMatcher

import json

In [177]:
neo4j_url = 'http://localhost:7474/'
user = 'neo4j'
pwd = 'xmxujie13'
graph = Graph(neo4j_url,  auth=(user, pwd), name = "neo4j")
node_matcher = NodeMatcher(graph)

In [178]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [179]:
# def add_file_operation(process_name, op, target_file):
#     # 如果节点不存在就需要创建
#     node1 = node_matcher.match("Process").where(name=process_name).first()
#     node2 = node_matcher.match("File").where(name=target_file).first()
    
#     if node1 is None:
#         node1 = Node("Process", name=process_name)
#         graph.create(node1)
    
#     if node2 is None:
#         node2 = Node("File", name=target_file)
#         graph.create(node2)
    
#     # build relationship
#     relation1 = Relationship(node1, op, node2)
    
#     graph.create(relation1)

In [180]:
# add_file_operation("program11", "openat", "secret.txt")
# add_file_operation("program11", "read", "secret.txt")
# add_file_operation("program11", "openat", "attacker/secret.txt")
# add_file_operation("program11", "write", "attacker/secret.txt")

读取数据, json格式进行转换

In [181]:
logFile = '../logs/auditbeat-20221130_filtered.json'

In [182]:
# read
fr = open(logFile, "r")
logs = [json.loads(line) for line in fr.readlines()]
fr.close()

print(len(logs))

19025


In [183]:
# 过滤只有program相关操作
filterLogs = []
filesLogs = []

for log in logs:
    ruletype = RuleType().parse_jsons(log['rule'])
    
    if ruletype.tag == "sys_access":
        filesLogs.append(json.loads(log['log']))



解析file操作

In [184]:

for log in filesLogs:
    
    pLocation = log['executable']
    
    if 'program' not in pLocation:
        continue
    
    fLocation = log['accessed_file']
    pName = pLocation.split('/')[-1]
    fName = fLocation.split('/')[-1]
    fType = 'File' if fName != 'secret.txt' else 'Secret'

    
    op = log['syscall']
    
    # additional information
    pid = log['pid'] 
    pUser = log['user']
    ts = log['timestamp']
    
    print(type(pUser))

    
    node1 = node_matcher.match("Process").where(name=pName).first()
    node2 = node_matcher.match("File").where(name=fName).first()
    
    if node1 is None:
        node1 = Node("Process", name=pName, location = pLocation, pid = pid, user = pUser['name'])
        graph.create(node1)
    
    if node2 is None:
        node2 = Node(fType, name=fName, location = fLocation)
        graph.create(node2)
        
    # build relationship
    relation1 = Relationship(node1, op, node2, timeStamp = ts)
    
    graph.create(relation1)
    

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

In [185]:
a = 'asd/asd/asdd'

print(a.split('/')[-1])

asdd
